In [1]:
import pandas as pd
import datetime
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import numpy as np
from matplotlib import pyplot as plt

In [2]:
#Build BackBeta DF
dfbb = pd.read_csv('retprc.csv')
dfbsp=pd.read_csv('sprtrn.csv')
dfbsp.rename(columns={'caldt':'datadate'},inplace=True)
dfbmerge=dfbb.merge(dfbsp, on = "datadate", how = "left") 
dfbmerge.dropna(inplace=True)
dfbmerge

,gvkey,datadate,indfmt,consol,popsrc,datafmt,curcd,costat,cik,LINKTYPE,LPERMNO,MthCalDt,MthPrc,MthRet,Ticker,IssuerNm,sprtrn
0,1447,2012-12-31,INDL,C,D,STD,USD,A,4962,LU,59176,2012-12-31,57.48,0.028265,AXP,AMERICAN EXPRESS CO,0.007068
1,1447,2013-12-31,INDL,C,D,STD,USD,A,4962,LU,59176,2013-12-31,90.73,0.057459,AXP,AMERICAN EXPRESS CO,0.023563
2,1447,2014-12-31,INDL,C,D,STD,USD,A,4962,LU,59176,2014-12-31,93.04,0.006709,AXP,AMERICAN EXPRESS CO,-0.004189
3,1447,2015-12-31,INDL,C,D,STD,USD,A,4962,LU,59176,2015-12-31,69.55,-0.029174,AXP,AMERICAN EXPRESS CO,-0.017530
6,1447,2018-12-31,INDL,C,D,STD,USD,A,4962,LU,59176,2018-12-31,95.32,-0.150975,AXP,AMERICAN EXPRESS CO,-0.091777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,3980,2015-09-30,INDL,C,D,STD,USD,A,1744489,LC,26403,2015-09-30,102.20,0.003141,DIS,DISNEY WALT CO,-0.026443
294,3980,2016-09-30,INDL,C,D,STD,USD,A,1744489,LC,26403,2016-09-30,92.86,-0.016938,DIS,DISNEY WALT CO,-0.001234
297,3980,2019-09-30,INDL,C,D,STD,USD,A,1744489,LC,26403,2019-09-30,130.32,-0.050561,DIS,DISNEY WALT CO,0.017181
298,3980,2020-09-30,INDL,C,D,STD,USD,A,1744489,LC,26403,2020-09-30,124.08,-0.059073,DIS,DISNEY WALT CO,-0.039228


In [8]:
dfbmerge['Year'] = pd.to_datetime(dfbmerge['datadate']).dt.year
results = []

grouped = dfbmerge.groupby(['Ticker', 'Year'])

for (ticker, year), group in grouped:
    X = group['sprtrn']
    y = group['MthRet']
    X = sm.add_constant(X)  
    model = sm.OLS(y, X).fit()
    
    beta = model.params['sprtrn']
    p_series = group['MthPrc']
    r_series = group['MthRet']

    p = p_series.iloc[0] if not p_series.empty else None
    r = r_series.iloc[0] if not r_series.empty else None
    results.append({'Ticker': gvkey, 'Year': year, 'Beta': beta,'Price': p,'Return': r})

df_beta = pd.DataFrame(results)
df_beta

,Ticker,Year,Beta,Price,Return
0,AAPL,2013,-0.722075,476.75,-0.021481
1,AAPL,2014,1.100490,100.75,-0.017073
2,AAPL,2015,0.825020,110.30,-0.021816
3,AAPL,2016,-53.082658,113.05,0.065504
4,AAPL,2019,4.246668,223.97,0.072962
...,...,...,...,...,...
222,WMT,2014,1.432116,74.68,-0.050959
223,WMT,2017,-1.925352,66.74,-0.034433
224,WMT,2018,1.415013,106.60,0.079494
225,WMT,2019,0.365652,95.83,0.028771


In [13]:
df_beta[df_beta['Year']==2015]

,Ticker,Year,Beta,Price,Return
2,AAPL,2015,0.825020,110.30,-0.021816
10,AMGN,2015,-0.435539,162.33,0.007635
18,AMZN,2015,-0.951626,675.89,0.016682
26,AXP,2015,1.664233,69.55,-0.029174
34,BA,2015,0.337307,144.59,-0.005913
42,CAT,2015,3.682601,67.96,-0.064556
57,CSCO,2015,2.173387,28.42,0.042907
65,CVX,2015,0.849572,89.96,-0.014893
72,DIS,2015,-0.118784,102.20,0.003141
80,GS,2015,2.939019,180.23,-0.051521


In [16]:
dfbmerge.info() # data leak NVIDIA and others are null

<class 'pandas.core.frame.DataFrame'>
Index: 227 entries, 0 to 299
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   gvkey     227 non-null    int64  
 1   datadate  227 non-null    object 
 2   indfmt    227 non-null    object 
 3   consol    227 non-null    object 
 4   popsrc    227 non-null    object 
 5   datafmt   227 non-null    object 
 6   curcd     227 non-null    object 
 7   costat    227 non-null    object 
 8   cik       227 non-null    int64  
 9   LINKTYPE  227 non-null    object 
 10  LPERMNO   227 non-null    int64  
 11  MthCalDt  227 non-null    object 
 12  MthPrc    227 non-null    float64
 13  MthRet    227 non-null    float64
 14  Ticker    227 non-null    object 
 15  IssuerNm  227 non-null    object 
 16  sprtrn    227 non-null    float64
 17  Year      227 non-null    int32  
dtypes: float64(3), int32(1), int64(3), object(11)
memory usage: 32.8+ KB


In [22]:
FundRatios = pd.read_csv('CRSP_COMPUSTAT_DJIA.csv')
FundRatios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 28 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   gvkey     353 non-null    int64  
 1   datadate  353 non-null    object 
 2   indfmt    353 non-null    object 
 3   consol    353 non-null    object 
 4   popsrc    353 non-null    object 
 5   datafmt   353 non-null    object 
 6   tic       353 non-null    object 
 7   curcd     353 non-null    object 
 8   ap        353 non-null    float64
 9   at        353 non-null    float64
 10  bkvlps    353 non-null    float64
 11  cogs      353 non-null    float64
 12  ebitda    353 non-null    float64
 13  gp        353 non-null    float64
 14  invt      353 non-null    float64
 15  lt        352 non-null    float64
 16  oancf     353 non-null    float64
 17  oiadp     353 non-null    float64
 18  rectr     331 non-null    float64
 19  revt      353 non-null    float64
 20  sale      353 non-null    float6

In [23]:


# Convert datadate to datetime and extract the year
FundRatios['datadate'] = pd.to_datetime(FundRatios['datadate']).dt.year
FundRatios['Year'] = FundRatios['datadate']
FundRatios['Ticker'] = FundRatios['tic']

# Sort DataFrame by Ticker and datadate
FundRatios = FundRatios.sort_values(by=['Ticker', 'datadate'])

# Drop unnecessary columns
columns_to_drop = ['datadate', 'gvkey', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcd', 'LINKTYPE', 'MthCalDt', 'LPERMNO']
FundRatios.drop(columns_to_drop, axis=1, inplace=True)

# Create lagged columns and fill NaN values with zero
lag_columns = ['invt', 'ap', 'at', 'rectr']
for col in lag_columns:
    FundRatios[f'lag_{col}'] = FundRatios.groupby('Ticker')[col].shift(1)
    FundRatios[f'lag_{col}'] = FundRatios[f'lag_{col}'].fillna(0)

# Calculate additional columns
FundRatios['TDTA'] = FundRatios['lt'] / FundRatios['at']
FundRatios['AP_TOV'] = FundRatios['cogs'] + (FundRatios['invt'] - FundRatios['lag_invt']) / ((FundRatios['ap'] + FundRatios['lag_ap']) / 2)
FundRatios['Inv_TOV'] = FundRatios['cogs'] / ((FundRatios['invt'] + FundRatios['lag_invt']) / 2)
FundRatios['change_in_ebitda'] = FundRatios.groupby('Ticker')['ebitda'].diff()
FundRatios['change_in_sales'] = FundRatios.groupby('Ticker')['sale'].diff()
FundRatios['ol'] = FundRatios['change_in_ebitda'] / FundRatios['change_in_sales']
FundRatios['ol'] = FundRatios['ol'].fillna(0)
FundRatios['AR_TOV'] = FundRatios['revt'] / ((FundRatios['rectr'] + FundRatios['lag_rectr']) / 2)
FundRatios['ROA'] = FundRatios['oiadp'] / ((FundRatios['at'] + FundRatios['lag_at']) / 2)
FundRatios['CFM'] = FundRatios['oancf'] / FundRatios['revt']
FundRatios['GPM'] = FundRatios['gp'] / FundRatios['revt']
FundRatios['btm'] = FundRatios['oancf'] / (FundRatios['prcl_f'] + ((FundRatios['prch_f'] - FundRatios['prcl_f']) / 2))

# Merge DataFrames
dfmerge = pd.merge(FundRatios, df_beta, on=['Year', 'Ticker'])
dfmerge = dfmerge.dropna(subset=['Year', 'Ticker','TDTA', 'AP_TOV', 'Inv_TOV', 'ol', 'AR_TOV', 'ROA', 'CFM','btm','GPM','Beta'])

# Reset index
dfmerge.reset_index(drop=True, inplace=True)

dfmerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 38 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tic               212 non-null    object 
 1   ap                212 non-null    float64
 2   at                212 non-null    float64
 3   bkvlps            212 non-null    float64
 4   cogs              212 non-null    float64
 5   ebitda            212 non-null    float64
 6   gp                212 non-null    float64
 7   invt              212 non-null    float64
 8   lt                212 non-null    float64
 9   oancf             212 non-null    float64
 10  oiadp             212 non-null    float64
 11  rectr             212 non-null    float64
 12  revt              212 non-null    float64
 13  sale              212 non-null    float64
 14  costat            212 non-null    object 
 15  prch_f            212 non-null    float64
 16  prcl_f            212 non-null    float64
 1

In [24]:
df = dfmerge.dropna(subset=['Year', 'Ticker','TDTA', 'AP_TOV', 'Inv_TOV', 'ol', 'AR_TOV', 'ROA', 'CFM','btm','GPM','Beta'])

# Reset index
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 38 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tic               212 non-null    object 
 1   ap                212 non-null    float64
 2   at                212 non-null    float64
 3   bkvlps            212 non-null    float64
 4   cogs              212 non-null    float64
 5   ebitda            212 non-null    float64
 6   gp                212 non-null    float64
 7   invt              212 non-null    float64
 8   lt                212 non-null    float64
 9   oancf             212 non-null    float64
 10  oiadp             212 non-null    float64
 11  rectr             212 non-null    float64
 12  revt              212 non-null    float64
 13  sale              212 non-null    float64
 14  costat            212 non-null    object 
 15  prch_f            212 non-null    float64
 16  prcl_f            212 non-null    float64
 1

In [27]:
# Output CSV File
df.to_csv('fund.csv', index=False)

In [ ]:
'''
CHECKING FOR 10 Year data runs
mask = (dfmerge1['Beta'] <= 10) & (dfmerge1['Beta'] >= -10) 

# Apply the mask to filter the DataFrame
df_filtered = dfmerge1[mask]
df_filtered = df_filtered.dropna()

# Reset index
df_filtered.reset_index(drop=True, inplace=True)
df_filtered.info()
'''

In [26]:
'''
ONLY USE FOR Modeling and Ticker is in GVKEY format
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df[['Ticker','Year','TDTA', 'AP_TOV', 'Inv_TOV', 'ol', 'AR_TOV', 'ROA', 'CFM','btm','GPM']].dropna(inplace=True)
    indices_to_keep = ~df[['Ticker','Year','TDTA', 'AP_TOV', 'Inv_TOV', 'ol', 'AR_TOV', 'ROA', 'CFM','btm','GPM','Beta']].isin([np.nan, np.inf, -np.inf]).any(axis=1)
    return df[['Ticker','Year','TDTA', 'AP_TOV', 'Inv_TOV', 'ol', 'AR_TOV', 'ROA', 'CFM','btm','GPM','Beta']][indices_to_keep].astype(np.float64)
dfc=clean_dataset(df)
dfc
'''

'\nONLY USE FOR Modeling and Ticker is in GVKEY format\ndef clean_dataset(df):\n    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"\n    df[[\'Ticker\',\'Year\',\'TDTA\', \'AP_TOV\', \'Inv_TOV\', \'ol\', \'AR_TOV\', \'ROA\', \'CFM\',\'btm\',\'GPM\']].dropna(inplace=True)\n    indices_to_keep = ~df[[\'Ticker\',\'Year\',\'TDTA\', \'AP_TOV\', \'Inv_TOV\', \'ol\', \'AR_TOV\', \'ROA\', \'CFM\',\'btm\',\'GPM\',\'Beta\']].isin([np.nan, np.inf, -np.inf]).any(axis=1)\n    return df[[\'Ticker\',\'Year\',\'TDTA\', \'AP_TOV\', \'Inv_TOV\', \'ol\', \'AR_TOV\', \'ROA\', \'CFM\',\'btm\',\'GPM\',\'Beta\']][indices_to_keep].astype(np.float64)\ndfc=clean_dataset(df)\ndfc\n'